# Airfoil experiments
All data is available in the repository

In [1]:
import sys
sys.path.append('../python')

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

from machine_learning import *
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

/usr/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Setup

In [2]:
qmc_points = np.loadtxt('../sobol_6_8000.txt')
qmc_points = qmc_points[1:].reshape((8000,6))

large_qmc_points = np.loadtxt('../sobol_6_131072.txt')
all_points = qmc_points.copy()
forces = np.array(np.loadtxt('../force_6_params.dat'))


N = min(qmc_points.shape[0], forces.shape[0])
qmc_points = qmc_points[:N,:]
forces  = forces[:N,:]


input_size=6
train_size=128
validation_size=128

epochs = 500000


airfoils_network = [12, 12, 10, 12, 10, 12, 10, 10, 12,1]


# One network per variable

In [ ]:
force_names=['Lift', 'Drag']
batch_sizes=[train_size]
train_sizes = [16, 32, train_size]

optimizers = {"SGD": keras.optimizers.SGD,
             "Adam": keras.optimizers.Adam}

losses = ["mean_squared_error", "mean_absolute_error"]

for optimizer in optimizers.keys():
    for loss in losses:
        display(HTML("<h1>{} with {}</h1>".format(optimizer, loss)))

        for batch_size in batch_sizes:
            tables = Tables.make_default()
            
            for (n, f) in enumerate(force_names):
                seed_random_number(random_seed)
                network_information = NetworkInformation(optimizer=optimizers[optimizer], epochs=epochs, 
                                                         network=airfoils_network, train_size=train_size,
                                                         validation_size=validation_size,
                                                        loss=loss, 
                                                        large_integration_points=large_qmc_points)
                
                output_information = OutputInformation(tables=tables, title=force_names[n],
                                                      short_title=force_names[n])
                showAndSave.prefix='airfoil_%s_%s_%s_ts_%d_bs_%d' %(optimizer, loss, f,batch_size, train_size)
                get_network_and_postprocess(qmc_points, forces[:,n+1], network_information = network_information,
                    output_information = output_information)
            
            showAndSave.prefix='airfoil_%s_%s_all_ts_%d_bs_%d' %(optimizer, loss, batch_size, train_size)
            tables.write_tables()
  

# As a function of training errors

In [ ]:
force_names=['Lift', 'Drag']
epochs = 500000

optimizers = {"SGD": keras.optimizers.SGD,
             "Adam": keras.optimizers.Adam}
loss = "mean_squared_error"
optimizer='SGD'
for n in range(len(force_names)):
    f=force_names[n]

    train_sizes = 2**np.arange(2, 10)
    errors = {}
    errors_comparison={}
    error_keys = ['prediction L2', 'wasserstein', 'mean', 'var']
    for error_key in error_keys:
        errors[error_key]=[]
        errors_comparison[error_key]=[]
    for train_size in train_sizes:
        tables = Tables.make_default()
        batch_size=train_size
        validation_size=train_size
        network_information = NetworkInformation(optimizer=optimizers[optimizer], epochs=epochs, 
                                                         network=airfoils_network, train_size=train_size,
                                                         validation_size=validation_size,
                                                        loss=loss, 
                                                        large_integration_points=None,
                                                tries=1)
                
        output_information = OutputInformation(tables=tables, title=force_names[n],
                                              short_title=force_names[n])
        showAndSave.prefix='airfoil_convergence_%s_%s_%s_ts_%d_bs_%d' %(optimizer, loss, f,batch_size, train_size)
        showAndSave.silent=True
        print_comparison_table.silent = True
        get_network_and_postprocess(qmc_points, forces[:,n+1], network_information = network_information,
            output_information = output_information)
        
        errors['prediction L2'].append(output_information.prediction_error[2])
        for stat in ['wasserstein', 'mean', 'var']:
            errors[stat].append(output_information.stat_error[stat])
            errors_comparison[stat].append(output_information.stat_error['%s_comparison'%stat])
    
    showAndSave.prefix = 'airfoils_convergence_%s_%d_%s' % (optimizer, epochs, f)
    showAndSave.silent=False
    for error_key in error_keys:
        error = errors[error_key]
        
        plt.loglog(train_sizes, error, '-o',label='DLQMC %s' % error_key)
        
        if 'prediction' not in error_key:
            comparison_error = errors_comparison[error_key]
            plt.loglog(train_sizes, comparison_error, '-o',label='QMC %s' % error_key)
        plt.legend()
        
        plt.xlabel('Training size')
        plt.ylabel("Error")
        plt.title("Error of %s as a function of training size" % error_key)
        showAndSave('convergence_%s' % error_key)

In [9]:
print(output_information.__dict__)

{'tables': <machine_learning.Tables object at 0x7f6fe1e2d860>, 'title': 'Lift', 'short_title': 'Lift', 'prediction_error': {1: 0.17019715383124276, 2: 0.19891631054709266}}
